<a href="https://www.kaggle.com/code/dewanrapa/dataset-clean-split?scriptVersionId=203388851" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import time
from joblib import Parallel, delayed
import multiprocessing

In [3]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Set the model to use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Check if using GPU
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

Using CPU


In [4]:
url = 'https://raw.githubusercontent.com/amanullahshah32/Review-Scraping/refs/heads/main/Dataset/all_app_reviews.csv'
df = pd.read_csv(url)

In [5]:
df.head()

,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date,developer_response,developer_response_date,appVersion,language_code,country_code,app_name
0,Google Play,d33f5fbd-e904-4c35-a29d-9bf66d393aa1,A Google user,NaN,nice,5,0,2024-10-10 12:50:00,NaN,NaN,NaN,en,us,com.twgbd.dims
1,Google Play,b1c90db8-c97e-401a-87db-f970ad5b7ce3,A Google user,NaN,good,5,0,2024-10-10 12:43:03,NaN,NaN,2.1.8,en,us,com.twgbd.dims
2,Google Play,fd85498c-702e-4bc7-8cde-70411670e214,A Google user,NaN,good,5,0,2024-10-10 12:35:00,NaN,NaN,2.1.8,en,us,com.twgbd.dims
3,Google Play,b8cc25c1-40c1-46d9-a238-bf0116de6ac4,A Google user,NaN,good,5,0,2024-10-10 12:18:52,NaN,NaN,2.1.8,en,us,com.twgbd.dims
4,Google Play,c0dfc093-493f-4906-80cb-1de222fbb02b,A Google user,NaN,good 😊,4,0,2024-10-10 11:39:51,NaN,NaN,2.1.8,en,us,com.twgbd.dims


In [6]:
df.shape

(99395, 14)

In [7]:
# Check for null values in 'review_description' and 'rating' columns
null_review_description = df['review_description'].isnull().sum()
null_rating = df['rating'].isnull().sum()

# Print the results
print(f"Null values in 'review_description': {null_review_description}")
print(f"Null values in 'rating': {null_rating}")


Null values in 'review_description': 39
Null values in 'rating': 0


In [8]:
df_cleaned = df.dropna(subset=['review_description'])

In [9]:
df_cleaned.head()

,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date,developer_response,developer_response_date,appVersion,language_code,country_code,app_name
0,Google Play,d33f5fbd-e904-4c35-a29d-9bf66d393aa1,A Google user,NaN,nice,5,0,2024-10-10 12:50:00,NaN,NaN,NaN,en,us,com.twgbd.dims
1,Google Play,b1c90db8-c97e-401a-87db-f970ad5b7ce3,A Google user,NaN,good,5,0,2024-10-10 12:43:03,NaN,NaN,2.1.8,en,us,com.twgbd.dims
2,Google Play,fd85498c-702e-4bc7-8cde-70411670e214,A Google user,NaN,good,5,0,2024-10-10 12:35:00,NaN,NaN,2.1.8,en,us,com.twgbd.dims
3,Google Play,b8cc25c1-40c1-46d9-a238-bf0116de6ac4,A Google user,NaN,good,5,0,2024-10-10 12:18:52,NaN,NaN,2.1.8,en,us,com.twgbd.dims
4,Google Play,c0dfc093-493f-4906-80cb-1de222fbb02b,A Google user,NaN,good 😊,4,0,2024-10-10 11:39:51,NaN,NaN,2.1.8,en,us,com.twgbd.dims


In [10]:
df_cleaned.shape

(99356, 14)

In [11]:
!pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=e34aa159aae40b83f994f0535d1cd611aa031f93d3ae5833796414e2b989fc2f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [22]:
from langdetect import detect

# Function to detect language of each review
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

# Apply the language detection function to the review_description column
df['detected_language'] = df['review_description'].apply(detect_language)

# Count the number of non-English reviews
non_english_reviews = df[df['detected_language'] != 'en'].shape[0]

# Print the result
print(f"Number of reviews not in English: {non_english_reviews}")

Number of reviews not in English: 34125


In [23]:
# Remove rows where the detected language is not English
df_cleaned = df[df['detected_language'] == 'en']

# Drop the 'detected_language' column as it's no longer needed
df_cleaned = df_cleaned.drop(columns=['detected_language'])

# Check the cleaned dataset
df_cleaned.head()

,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date,developer_response,developer_response_date,appVersion,language_code,country_code,app_name
9,Google Play,f18a5ffe-872c-4f63-bf38-2c0eb5a52170,A Google user,NaN,The apps useful. Best service,5,0,2024-10-10 10:36:15,NaN,NaN,2.1.8,en,us,com.twgbd.dims
13,Google Play,79ea060a-e2fb-44e5-a2b1-8525f21b5323,A Google user,NaN,very interestin,5,0,2024-10-10 06:56:36,NaN,NaN,2.1.8,en,us,com.twgbd.dims
14,Google Play,53f9e22f-cbb4-48ee-9e1b-a339ad4760ca,A Google user,NaN,good apps,5,0,2024-10-10 06:52:01,NaN,NaN,2.1.8,en,us,com.twgbd.dims
15,Google Play,4170d3ff-6f9f-45f6-910f-04a131b4a081,A Google user,NaN,very nice app,5,0,2024-10-10 06:46:15,NaN,NaN,2.1.8,en,us,com.twgbd.dims
17,Google Play,16e82105-ecb2-45ca-be80-9283490047ed,A Google user,NaN,good apps,5,0,2024-10-10 05:35:54,NaN,NaN,2.1.8,en,us,com.twgbd.dims


In [24]:
df_cleaned.shape

(65270, 14)

In [26]:
df_cleaned.to_csv('cleaned_dataset.csv', index=False)